## Dataset


#### 1. Tourism-Focused Criminal Data Analysis: Preprocessing Barcelona's Crime Dataset: 

This Jupyter notebook focuses on preparing criminal data retrieved from an API for analysis in the tourism sector. The process involves extracting relevant information and structuring it appropriately to facilitate further analysis.


In [47]:
# Imports
import pandas as pd
import requests
import io

In [48]:
dataset_urls = {
    'criminal_acts': 'https://analisi.transparenciacatalunya.cat/resource/y48r-ae59.json?$limit=50000000'  
}

In [49]:
criminal_dataset = requests.get(dataset_urls['criminal_acts']).content
criminal_dataset = pd.read_json(io.StringIO(criminal_dataset.decode('utf-8')))
criminal_dataset.head(3)

,any,n_m_mes,nom_mes,regi_policial_rp,rea_b_sica_policial_abp,prov_ncia,comarca,municipi,tipus_de_lloc_dels_fets,canal_dels_fets,tipus_de_fet,t_tol_del_fet_codi_penal,tipus_de_fet_codi_penal_o,mbit_fet,nombre_fets_o_infraccions,nombre_v_ctimes
0,2021,12,Desembre,RP Metropolitana Nord,ABP Badalona,Barcelona,Barcelonès,Badalona,Via pública urbana,Presencial,Delictes,De les lesions,Lesions,Sexisme,1,1.0
1,2021,6,Juny,RP Central,ABP Anoia,Barcelona,Anoia,Resta municipis ABP Anoia,Habitatge - Escala/portal finca/replà,Presencial,Delictes,Delictes contra la llibertat,Amenaces,Sexisme,1,1.0
2,2021,6,Juny,RP Metropolitana Barcelona,ABP Nou Barris,Barcelona,Barcelonès,Barcelona,Via pública urbana,Presencial,Delictes,Delictes contra la llibertat,Amenaces,Orientació política,1,1.0


## Dataset Annotations

The dataset comprises the following columns:

- any, num_mes, nom_mes, regio_policial, area_basica_policial, provincia, comarca, municipi, tipus_de_lloc_dels_fets, canal_dels_fets, tipus_de_fet, titol_del_fet_codi_penal, tipus_de_fet_codi_penal, ambit_fet, nombre_fets_o_infraccions, nombre_victimes.

During the preprocessing stage, the following actions were undertaken:

- Renamed columns with accent-related export issues.
- Filtered data to include only entries pertaining to the province of Barcelona.
- Eliminated entries from the tipus_de_lloc_dels_fets column that are not relevant to tourism.

All preprocessing steps are documented within the `preprocessing_criminal_dataset` function for transparency and reproducibility.

In [50]:
spaces_not_in_domain = ["Empr. tèxtil, calçat i complementaris", "Taller de Vehicles_CC", "NaN", "Obra en construcció", "Sindicat", "Centre de Menors", "Partit polític", "Empresa de la construcció", "Centre d'Atenció Primària", 'Associació de veïns', "Institut d'Educació", "Empreses de productes d'alimentació/begudes", "Dependències",  "Escola",  "Oficines/Despatxos professionals",  "Centre de dia/Residència persones grans"]

def preprocessing_criminal_dataset(dataset, list_not_domain=spaces_not_in_domain):
    dataset = dataset.rename(columns={'n_m_mes': 'num_mes', 
                                                    'regi_policial_rp': 'regio_policial', 
                                                    'rea_b_sica_policial_abp': 'area_basica_policial', 
                                                    'prov_ncia': 'provincia', 
                                                    't_tol_del_fet_codi_penal': 'titol_del_fet_codi_penal', 
                                                    'tipus_de_fet_codi_penal_o': 'tipus_de_fet_codi_penal', 
                                                    'mbit_fet': 'ambit_fet', 
                                                    'nombre_v_ctimes': 'nombre_victimes'})
    dataset = dataset.loc[dataset["provincia"] == 'Barcelona']

    mask = dataset['tipus_de_lloc_dels_fets'].apply(lambda x: any(space in str(x) for space in list_not_domain))
    dataset = dataset[~mask]
    dataset= dataset.dropna(subset=['tipus_de_lloc_dels_fets'])

    return criminal_dataset

In [51]:
criminal_dataset = preprocessing_criminal_dataset(criminal_dataset)

In [52]:
criminal_dataset.head(3)

,any,n_m_mes,nom_mes,regi_policial_rp,rea_b_sica_policial_abp,prov_ncia,comarca,municipi,tipus_de_lloc_dels_fets,canal_dels_fets,tipus_de_fet,t_tol_del_fet_codi_penal,tipus_de_fet_codi_penal_o,mbit_fet,nombre_fets_o_infraccions,nombre_v_ctimes
0,2021,12,Desembre,RP Metropolitana Nord,ABP Badalona,Barcelona,Barcelonès,Badalona,Via pública urbana,Presencial,Delictes,De les lesions,Lesions,Sexisme,1,1.0
1,2021,6,Juny,RP Central,ABP Anoia,Barcelona,Anoia,Resta municipis ABP Anoia,Habitatge - Escala/portal finca/replà,Presencial,Delictes,Delictes contra la llibertat,Amenaces,Sexisme,1,1.0
2,2021,6,Juny,RP Metropolitana Barcelona,ABP Nou Barris,Barcelona,Barcelonès,Barcelona,Via pública urbana,Presencial,Delictes,Delictes contra la llibertat,Amenaces,Orientació política,1,1.0


In [55]:
# Identificar columnas con valores NaN
columnas_con_nan = criminal_dataset.columns[criminal_dataset.isna().any()].tolist()
valores_nan_por_columna = criminal_dataset.isna().sum()


# Imprimir las columnas con valores NaN
print("Las siguientes columnas tienen valores NaN:")
print(valores_nan_por_columna)
# Imprimir la cantidad de valores NaN por columna

Las siguientes columnas tienen valores NaN:
any                             0
n_m_mes                         0
nom_mes                         0
regi_policial_rp                0
rea_b_sica_policial_abp         0
prov_ncia                       0
comarca                         0
municipi                        0
tipus_de_lloc_dels_fets         1
canal_dels_fets                 0
tipus_de_fet                    0
t_tol_del_fet_codi_penal        0
tipus_de_fet_codi_penal_o       0
mbit_fet                        0
nombre_fets_o_infraccions       0
nombre_v_ctimes              1426
dtype: int64


In [59]:
criminal_dataset = criminal_dataset.dropna(subset=['tipus_de_lloc_dels_fets'])